<a href="https://colab.research.google.com/github/CobaMasa/RStan2PyStan/blob/master/chapter8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Chapeter8: Hierarchical model
**Tips**
- Some features (i.g)effects drived from Individuals or places) that are NOT OBSERVED in experiments, can be included within hierarchical models.
- These features are usually considered to be generated from $Normal(0, \sigma)$ and we can estimate the parameter $\sigma$from data.
- Even in the era of bigdata, or VUCA, sometime we can't collect enought data which stand for specific effects from individuals or groups, that's the time hierarchical models are good to be chosen.

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import pystan

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## Wihtout considering groups' diff
$$
\begin{eqnarray}
    Y[n]&〜&y_{base}[n]+\epsilon[n]\\
    y_{base}[n]&〜&a+bX[n]\\
    \epsilon[n]&〜&Normal(0, \sigma_{Y})\\
\end{eqnarray}
$$
which means...

$$\begin{eqnarray}Y[n]&〜&Normal(a+bX[n], \sigma_{Y})\end{eqnarray}$$